# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui
variaveis = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
             'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
novo_sinasc = sinasc[variaveis].copy()
novo_sinasc.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [4]:
# 4) seu código aqui
novo_sinasc.dropna(subset=['APGAR5'], inplace=True)
novo_sinasc.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [5]:
# 5) seu código aqui
novo_sinasc.loc[novo_sinasc['ESTCIVMAE'] == 9] = pd.NA
novo_sinasc.dropna(subset='ESTCIVMAE', inplace=True)
print(novo_sinasc['CONSULTAS'].value_counts()) ## CONSULTAS não possui código 9
novo_sinasc.isna().sum()

CONSULTAS
4.0    18285
3.0     5852
2.0     1337
1.0      942
9.0       50
Name: count, dtype: int64


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         242
QTDFILVIVO    1526
GESTACAO      1183
GRAVIDEZ        73
CONSULTAS        0
APGAR5           0
dtype: int64

In [6]:
# 6) Seu código aqui
print(novo_sinasc['QTDFILVIVO'].isna().sum())
novo_sinasc['QTDFILVIVO'].fillna(0, inplace=True)
novo_sinasc['QTDFILVIVO'].value_counts()

1526


QTDFILVIVO
0.0     10674
1.0      9224
2.0      4376
3.0      1331
4.0       467
5.0       210
6.0       103
7.0        39
8.0        22
9.0         8
10.0        6
12.0        3
11.0        1
30.0        1
14.0        1
Name: count, dtype: int64

In [25]:
# 7) seu código aqui
print(novo_sinasc.isna().sum())
novo_sinasc['ESCMAE'].fillna(method='ffill', inplace=True)
# novo_sinasc.loc[novo_sinasc['ESCMAE'].isna()] = novo_sinasc['ESCMAE'].fillna(method='ffill')
# novo_sinasc[novo_sinasc['ESCMAE'] == 'ffill']
novo_sinasc['ESCMAE'].value_counts()
# novo_sinasc.loc[novo_sinasc['GESTACAO'].isna()]# = novo_sinasc['GESTACAO'].fillna(novo_sinasc['GESTACAO'].median(), inplace=True)
# novo_sinasc.loc[novo_sinasc['GRAVIDEZ'].isna()]# = novo_sinasc['GRAVIDEZ'].fillna(novo_sinasc['GRAVIDEZ'].median(), inplace=True)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1183
GRAVIDEZ        73
CONSULTAS        0
APGAR5           0
dtype: int64


ESCMAE
8 a 11 anos        15476
12 anos ou mais     5743
4 a 7 anos          4699
1 a 3 anos           509
Nenhuma               39
Name: count, dtype: int64

In [183]:
# 8) seu código aqui


In [ ]:
# 9) seu código aqui
